In [36]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import pandas as pd
import numpy as np

In [2]:
# Ruta del archivo CSV
ruta_archivo = 'dataset_clean.csv'

# Cargar el CSV como un DataFrame
df = pd.read_csv(ruta_archivo)

# Mostrar las primeras filas
df.head()

C:\Users\USER\AppData\Local\Temp\ipykernel_9244\3546070593.py:5: DtypeWarning: Columns (780) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(ruta_archivo)


,Pixel1,Pixel2,Pixel3,Pixel4,Pixel5,Pixel6,Pixel7,Pixel8,Pixel9,Pixel10,...,Pixel772,Pixel773,Pixel774,Pixel775,Pixel776,Pixel777,Pixel778,Pixel779,Pixel780,Categoria
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2


In [13]:
# Cargar el contenido en listas
X = np.array([[int(x) for x in list(fila)[:-1]] for _,fila in df.iterrows()])
y = np.array([str(i) for i in list(df["Categoria"])])
X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [14]:
# Dividir el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print('X_train', X_train.shape)
print('X_test' , X_test.shape )
print('y_train', y_train.shape)
print('y_test' , y_test.shape )

X_train (12093, 780)
X_test (3024, 780)
y_train (12093,)
y_test (3024,)


In [16]:
model = RandomForestClassifier()

In [17]:
# Definir la cuadrícula de hiperparámetros
param_grid = {
    'n_estimators': [10, 50, 100],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10]
}
param_grid

{'n_estimators': [10, 50, 100],
 'max_depth': [None, 10, 20],
 'min_samples_split': [2, 5, 10]}

In [18]:
# Configurar la búsqueda en cuadrícula
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy')
grid_search

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [None, 10, 20],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [10, 50, 100]},
             scoring='accuracy')

In [19]:
# Ajustar el modelo y buscar los mejores hiperparámetros
grid_search.fit(X_train, y_train)
print('end')

end


In [20]:
# Mostrar los mejores hiperparámetros
print(f"Mejores hiperparámetros: {grid_search.best_params_}")

Mejores hiperparámetros: {'max_depth': None, 'min_samples_split': 5, 'n_estimators': 10}


In [21]:
# Evaluar el modelo con los mejores hiperparámetros en el conjunto de prueba
best_model = grid_search.best_estimator_
best_model

RandomForestClassifier(min_samples_split=5, n_estimators=10)

In [37]:
y_pred_dt = best_model.predict(X_test)
# Métricas para árbol de decisión
print("\nResultados para Árbol de Decisión:")
print("Accuracy:", accuracy_score(y_test, y_pred_dt))
print("Reporte de clasificación:")
print(classification_report(y_test, y_pred_dt))


Resultados para Árbol de Decisión:
Accuracy: 0.9976851851851852
Reporte de clasificación:
              precision    recall  f1-score   support

           2       1.00      1.00      1.00       107
           3       1.00      1.00      1.00        75
           4       1.00      1.00      1.00        89
           5       1.00      1.00      1.00        93
           6       0.99      1.00      0.99        82
           7       1.00      1.00      1.00        99
           8       0.99      1.00      0.99        98
           9       1.00      0.99      0.99       100
           A       1.00      1.00      1.00        94
           B       0.98      1.00      0.99        98
           C       1.00      0.99      0.99        89
           D       1.00      1.00      1.00        79
           E       1.00      1.00      1.00        85
           F       1.00      1.00      1.00        79
           G       1.00      1.00      1.00        84
           H       1.00      1.00      1.00 

In [23]:
a1 = best_model.predict(X_test)
for i in range(len(y_test)):
    a = a1[i]
    b = y_test[i]
    if not a == b:
        print(i, a, b)


396 L I
825 X K
1526 B M
2042 B P
2235 W Q
2592 8 9
2598 6 C


<h2>Modelo de red neuronal</h2>

In [30]:
from sklearn.neural_network import MLPClassifier
from imblearn.over_sampling import SMOTE

In [31]:
# Aplicar SMOTE al conjunto de entrenamiento
smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

# Verificar el balanceo
print("Distribución de clases antes del balanceo:")
print(pd.Series(y_train).value_counts())

print("\nDistribución de clases después del balanceo:")
print(pd.Series(y_train_balanced).value_counts())

Distribución de clases antes del balanceo:
G    390
Y    388
8    387
L    387
U    385
X    381
Q    380
T    378
B    377
7    377
S    376
4    376
F    375
P    374
I    369
C    368
2    367
D    366
3    365
Z    365
V    364
W    363
5    361
J    360
6    356
E    354
H    352
N    349
K    346
R    346
M    340
9    339
A    332
Name: count, dtype: int64

Distribución de clases después del balanceo:
C    390
G    390
N    390
Z    390
K    390
5    390
3    390
H    390
L    390
8    390
W    390
T    390
B    390
7    390
Y    390
2    390
Q    390
S    390
A    390
F    390
4    390
M    390
D    390
E    390
X    390
9    390
V    390
U    390
J    390
P    390
I    390
R    390
6    390
Name: count, dtype: int64


In [32]:
mlp = MLPClassifier(random_state=42, max_iter=500)

In [33]:
# Espacio de búsqueda de hiperparámetros
param_grid_mlp = {
    'hidden_layer_sizes': [(50,), (100,), (50, 50)],
    'activation': ['relu', 'tanh'],
    'learning_rate': ['constant', 'adaptive'],
    'alpha': [0.0001, 0.001],
}

In [34]:
# Ajustar modelo con GridSearchCV
grid_mlp = GridSearchCV(mlp, param_grid_mlp, cv=5, scoring='accuracy', n_jobs=-1)
grid_mlp.fit(X_train_balanced, y_train_balanced)

GridSearchCV(cv=5, estimator=MLPClassifier(max_iter=500, random_state=42),
             n_jobs=-1,
             param_grid={'activation': ['relu', 'tanh'],
                         'alpha': [0.0001, 0.001],
                         'hidden_layer_sizes': [(50,), (100,), (50, 50)],
                         'learning_rate': ['constant', 'adaptive']},
             scoring='accuracy')

In [35]:
best_mlp = grid_mlp.best_estimator_
print("Mejores parámetros de MLP:", grid_mlp.best_params_)

Mejores parámetros de MLP: {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (50,), 'learning_rate': 'constant'}


In [38]:
y_pred_mlp = best_mlp.predict(X_test)
# Métricas para red neuronal
print("Resultados para Red Neuronal:")
print("Accuracy:", accuracy_score(y_test, y_pred_mlp))
print("Reporte de clasificación:")
print(classification_report(y_test, y_pred_mlp))

Resultados para Red Neuronal:
Accuracy: 0.9973544973544973
Reporte de clasificación:
              precision    recall  f1-score   support

           2       1.00      1.00      1.00       107
           3       1.00      1.00      1.00        75
           4       1.00      1.00      1.00        89
           5       1.00      1.00      1.00        93
           6       1.00      1.00      1.00        82
           7       1.00      1.00      1.00        99
           8       0.99      1.00      0.99        98
           9       1.00      0.99      0.99       100
           A       1.00      1.00      1.00        94
           B       0.99      1.00      0.99        98
           C       1.00      1.00      1.00        89
           D       1.00      1.00      1.00        79
           E       1.00      0.99      0.99        85
           F       0.99      1.00      0.99        79
           G       1.00      1.00      1.00        84
           H       1.00      1.00      1.00       

In [39]:
a1 = best_mlp.predict(X_test)
for i in range(len(y_test)):
    a = a1[i]
    b = y_test[i]
    if not a == b:
        print(i, a, b)

228 F E
396 L I
825 X K
1465 Y T
1526 B M
2235 W Q
2480 Y T
2592 8 9


In [24]:
from collections import Counter
print("Distribución de clases en y_train:", Counter(y_train))

Distribución de clases en y_train: Counter({np.str_('G'): 390, np.str_('Y'): 388, np.str_('L'): 387, np.str_('8'): 387, np.str_('U'): 385, np.str_('X'): 381, np.str_('Q'): 380, np.str_('T'): 378, np.str_('B'): 377, np.str_('7'): 377, np.str_('S'): 376, np.str_('4'): 376, np.str_('F'): 375, np.str_('P'): 374, np.str_('I'): 369, np.str_('C'): 368, np.str_('2'): 367, np.str_('D'): 366, np.str_('Z'): 365, np.str_('3'): 365, np.str_('V'): 364, np.str_('W'): 363, np.str_('5'): 361, np.str_('J'): 360, np.str_('6'): 356, np.str_('E'): 354, np.str_('H'): 352, np.str_('N'): 349, np.str_('K'): 346, np.str_('R'): 346, np.str_('M'): 340, np.str_('9'): 339, np.str_('A'): 332})
